In [1]:
# paquete para leer archivos html
import bs4
# urllib es unpaquete para acceder a contenido de paginas por medio de URL
from urllib.request import urlopen,Request
import urllib
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd

In [2]:
## Funciones auxiliares
def get_soup(url,pars):
    """
    Retorna una sopa de caracteres de la pagina leída en formato especificado
    args:
        url: url de la pagina
        pars: string de tipo de formato en el cual se va a leer la pagina (html o xml)
    returns:
        objeto de tipo BeautifulSoup con su contenido
    """
    #myURL = url
    uClient = urlopen(url)
    page = uClient.read()
    uClient.close()
    page_soup = BeautifulSoup(page,pars)
    return page_soup


def get_opiniones(url_base,tag,atributo,valor,n_pags,conn):
    """
    Consigue todas las opiniones de una URL del foro
    
    args:
        url_base: url base que se concatena con el indice de la pagina
        tag: etiqueta del html donde se encuentra el texto buscado
        atributo: atributo del objeto con etiqueta tag
        valor: valor del atributo
        n_pags: numero maximo de paginas de comentarios maximo a considerar
    
    returns:
        lista con opiniones
    """
    print(url_base)
    opiniones = []
    print("Explorando paginas...")
    for i in range(1,n_pags+1):
        #print("Pagina: ",i)
        URL = url_base + conn + str(i) # "?page=" o 
        soup = get_soup(URL,"html.parser")
        ops_pg = soup.findAll(tag,{atributo:value})
        for op in ops_pg:
            opiniones.append(op.text.replace('\n',''))
        #print("========================================")    
    print("Terminado!")
    return opiniones


## BBVA

In [3]:
url_bbva = "https://www.amazon.com.mx/BBVA-Bancomer-BBVA-Bancomer/product-reviews/B07K7X7JFP"
soup = get_soup(url_bbva,"html.parser")
# https://www.amazon.com.mx/BBVA-Bancomer-BBVA-Bancomer/product-reviews/B07K7X7JFP

In [4]:
tag1 = "a"
atributo1 = "data-hook"
valor1 = "review-title"

tag2 = "span"
atributo2 = "class"
valor2 = "a-size-base review-text review-text-content"


opiniones_bbva = []

ops_bbva_1 = soup.findAll(tag1,{atributo1:valor1})
ops_bbva_2 = soup.findAll(tag2,{atributo2:valor2})

In [5]:
# Correr una sola vez sino desde celdas de arriba
for i in range(len(ops_bbva_1)):
    opiniones_bbva.append(ops_bbva_1[i].text + ". " + ops_bbva_2[i].text)
    
opiniones_bbva = [o.replace('\n','') for o in opiniones_bbva]

In [6]:
opiniones_bbva

['Me encanta! Puedo obtener el tipo de cambio rápidamente..   Puedo obtener el tipo de cambio rápidamente y me da tips financieros.',
 'BBVA el banco digital.   Excelente skill, puedes mantenerte al día de indicadores bancarios de inmediato. Se debería poder hacer parte de las rutinas y deberían agregar más funciones',
 'mala ubicacion de sucursales.   Solo reconoce algunos c.p. En la mayoria te dice que no hay sucursal. Ya da el tipo de cambio en compra y venta. Cuando ya no quieres ayuda, no reconoce los comandos no,ni alto, siempre responde que no entiende lo que deseas...',
 'Indispensable para usuarios BBVA Bancomer.   Un muy buen Skill para los usuarios de BBVA Bancomer, con servicios necesarios y eficientes.',
 'Interesante e inovador.   Desde la aplicación es super fácil e intuitivo, jamas pensé que podría interactuar así con un banco',
 'código postal.   no reconoce ninguna código postal para ubicar cajeros o sucursales.',
 'Me gusto.   Espero que pongan mas funciones',
 'pedi

## Citibanamex

In [7]:
URL_base_citibmx = "https://es.trustpilot.com/review/www.banamex.com"
tag = "p"
atributo = "class"
value = "review-content__text"
num_pags = 2
opiniones_citibmx = get_opiniones(URL_base_citibmx, tag, atributo, value, num_pags,"?page=")

https://es.trustpilot.com/review/www.banamex.com
Explorando paginas...
Terminado!


In [9]:
print(len(opiniones_citibmx))
opiniones_citibmx = [opiniones_citibmx[i].strip() for i in range(len(opiniones_citibmx))]

22


## Santander

In [10]:
# Con Rankia
def find_Rankia_opinions(banco,url_base,pags_range,conn):
    """
    Encuentra las opiniones relacionadas con el banco especifico en la pagina de Rankia referente al banco
    args:
        banco: nombre del banco
        url_base: url donde hay opiniones de bancos en la pagina Rankia
        pags_range: rango de indice de paginas en las cuales se va a examinar
        conn: conector para explorar paginas
    returns:
        lista con opiniones referentes al banco especificado
    """
    print("Explorando paginas...")
    opiniones = []
    for i in pags_range:
        URL = url_base + conn + str(i)
        page = Request(URL,headers={'User-Agent': 'Mozilla/5.0'}) 
        infile= urllib.request.urlopen(page).read()
        soup = BeautifulSoup(infile,"html.parser")#get_soup(url_base_sant,"html.parser")
        ops_pg = soup.findAll("p")
        for j in range(len(ops_pg)):
            if banco in ops_pg[j].text:
                opiniones.append(ops_pg[j].text)
    print("Terminado!")
    return opiniones
        

In [11]:
banco = "Santander"
URL_base_sant = "https://www.rankia.mx/foros/bancos-cajas/temas/1764910-cual-mejor-banco-mexico"
pags_range = range(8,14)
conn = "?page="
opiniones_sant = find_Rankia_opinions(banco,URL_base_sant,pags_range,conn)

Explorando paginas...
Terminado!


## Pasando los datos a dataframe

In [14]:
opiniones_MEX = np.array(opiniones_bbva + opiniones_citibmx + opiniones_sant)
df_mex = pd.DataFrame(opiniones_MEX,columns=['opinion'])
df_mex['banco'] = np.array(len(opiniones_bbva)*["BBVA"]+len(opiniones_citibmx)*["Citibanamex"]+len(opiniones_sant)*["Santander"])
df_mex["pais"] = (len(opiniones_bbva)+len(opiniones_citibmx)+len(opiniones_sant))*["México"]

In [17]:
df_mex.head()

,opinion,banco,pais
0,Me encanta! Puedo obtener el tipo de cambio rá...,BBVA,México
1,"BBVA el banco digital. Excelente skill, pued...",BBVA,México
2,mala ubicacion de sucursales. Solo reconoce ...,BBVA,México
3,Indispensable para usuarios BBVA Bancomer. U...,BBVA,México
4,Interesante e inovador. Desde la aplicación ...,BBVA,México


In [18]:
# Exportacion a excel
df_mex.to_excel("opiniones_mexico.xlsx")